In [ ]:
# from google.colab import drive # 挂载谷歌云盘
# drive.mount('/content/drive')
# !nvidia-smi # 显示显卡信息
# ''' 符号%代表一直生效，！代表执行完立马结束，不会生效，所以进入目录用% '''
# %cd /content/drive/MyDrive/timeSerise
# ''' 支持的 常用命令1.ls  2.wget  3.gdoint(int(int(int(w))))n  4.mkdir  5.pwd '''
# !ls
# !pip install patool
# !pip install sktime
# !pip install reformer_pytorch

In [2]:
import random
import numpy as np
import torch
from exp.exp_long_term_forecasting import Exp_Long_Term_Forecast
# from exp.exp_imputation import Exp_Imputation
# from exp.exp_short_term_forecasting import Exp_Short_Term_Forecast
# from exp.exp_anomaly_detection import Exp_Anomaly_Detection
# from exp.exp_classification import Exp_Classification
# from data_provider.data_creat import *
# import akshare as ak
from torch.utils.tensorboard import SummaryWriter

In [3]:
class Args:
    '''基本配置'''
    # 选项：[long_term_forecast, short_term_forecast, imputation, classification, anomaly_detection]')
    task_name = 'long_term_forecast'
    is_training = 0
    model_id = 'Stock_test'
    # 模型名称，选项：[Autoformer, Transformer, TimesNet]
    model = 'TimesNet'
    
    '''数据加载'''
    # 数据集类型,选项：[ETTh1,ETTh2,ETTm1,ETTm2,custom,m4,PSM,MSL,SMAP,SMD,SWAT,UEA]
    data = 'custom'
    root_path = './dataset/Stock/'
    data_path = 'all_data_scaled.csv'
    # 预测任务 M:多变量预测多变量, S:单变量预测单变量, MS:多变量预测单变量
    features = 'MS'
    # 目标列名，S或MS任务中的目标特征
    target = 'OT'
    # 时间采集粒度，选项：[s:秒, t:分钟, h:小时, d:天, b:工作日, w:周, m:月]
    freq = 'd'
    # 模型检查点的位置
    checkpoints = './checkpoints/'

    '''预测任务'''
    # 输入序列长度,这是用于模型训练的输入序列的长度
    seq_len = 60
    # 开始标记长度,这是模型输出目标中有标签数据的长度，类似于滑动窗口的长度
    label_len = 20
    # 预测序列长度
    pred_len = 1
    # 季节模式（针对M4数据集）
    seasonal_patterns = 'Monthly'
    inverse = False    # 反转输出数据

    '''插补任务'''
    # 插补任务中数据丢失率
    mask_rate = 0.25

    '''异常检测任务'''
    # 异常检测中异常点占比
    anomaly_ratio = 0.25

    '''模型定义'''
    # TimesBlock 中傅里叶变换,频率排名前k个周期
    top_k = 5
    # Inception 中卷积核个数
    num_kernels = 6
    # encoder 输入特征数
    enc_in = 38
    # decoder 输入特征数
    dec_in = 38
    # 输出通道数
    c_out = 38
    # 线性层隐含神经元个数
    d_model = 32
    # FFN 层隐含神经元个数
    d_ff = 32
    # 多头注意力机制
    n_heads = 8
    # encoder 层数
    e_layers = 2
    # decoder 层数
    d_layers = 1
    # 滑动窗口长度
    moving_avg = 20
    # 对 Q 进行采样，对 Q 采样的因子数
    factor = 3
    # 是否下采样操作 pooling
    distil = True
    # dropout 率
    dropout = 0.1
    # 时间特征嵌入方式,选项：[timeF, fixed, learned]
    embed = 'timeF'
    # 激活函数类型
    activation = 'gelu'
    # 是否输出 attention
    output_attention = False

    '''优化'''
    # 并行核心数
    num_workers = 10
    # 实验轮数
    itr = 1
    # 训练迭代次数
    train_epochs = 500
    # batch size 大小
    batch_size = 256
    # early stopping 机制容忍次数
    patience = 53
    # 学习率
    learning_rate = 0.0001
    # 实验描述
    des = 'stock'
    # 损失函数
    loss = 'MSE'
    # 学习率下降策略
    lradj = 'type1'
    # 使用混合精度训练
    use_amp = False

    '''GPU'''
    # 使用 gpu
    use_gpu = True
    gpu = 0
    # 使用多个 gpus
    use_multi_gpu = False
    # 多 gpu 的设备 id
    devices = '0,1,2,3'

    '''去平稳化投影仪参数'''
    # 投影仪的隐藏层维度（列表）
    p_hidden_dims = [128, 128]
    # 投影仪中的隐藏层数
    p_hidden_layers = 2


# 创建参数对象
args = Args()

# 设置随机种子以确保结果可重现
fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)


# 获取列数
num_columns = 33
# args.des = NUM
args.enc_in = num_columns - 1
args.dec_in = num_columns - 1
args.c_out = num_columns - 1

In [4]:
# 检查并设置 GPU
args.use_gpu = torch.cuda.is_available() and args.use_gpu
if args.use_gpu:
    print("使用 GPU.")
    total_cuda_devices = torch.cuda.device_count()  # 获取系统中可用的 GPU 总数
    print(f"系统中总共有 {total_cuda_devices} 个 CUDA 设备可用。")
    if args.use_multi_gpu:
        args.devices = args.devices.replace(' ', '')
        device_ids = args.devices.split(',')
        args.device_ids = [int(id_) for id_ in device_ids]
        args.gpu = args.device_ids[0]
        
        # 打印多 GPU 使用情况
        print(f"使用多个GPU: {args.device_ids}")
        device = torch.device(f"cuda:{args.gpu}" if args.use_gpu else "cpu")
        print(f"Primary GPU (cuda:{args.gpu}) is in use.")
    else:
        args.gpu = 0
        device = torch.device("cuda" if args.use_gpu else "cpu")
        print("使用单个 GPU.")
else:
    device = torch.device("cpu")
    print("使用 CPU.")

# 选择合适的实验类
if args.task_name == 'long_term_forecast':
    Exp = Exp_Long_Term_Forecast
# elif args.task_name == 'short_term_forecast':
#     Exp = Exp_Short_Term_Forecast
# elif args.task_name == 'imputation':
#     Exp = Exp_Imputation
# elif args.task_name == 'anomaly_detection':
#     Exp = Exp_Anomaly_Detection
# elif args.task_name == 'classification':
#     Exp = Exp_Classification
else:
    Exp = Exp_Long_Term_Forecast  # 默认情况

# 进行训练和测试
if args.is_training:
    for ii in range(args.itr):
        # 实验记录设置
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.task_name,
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des, ii)

        exp = Exp(args)  # 设置实验
        print('>>>>>>>开始训练 : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>测试 : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)
        if args.use_gpu:
            torch.cuda.empty_cache()
else:
    ii = 0
    setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
        args.task_name,
        args.model_id,
        args.model,
        args.data,
        args.features,
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.factor,
        args.embed,
        args.distil,
        args.des, ii)

    exp = Exp(args)  # 设置实验
    print('>>>>>>>测试 : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    exp.test(setting, test=1)
    if args.use_gpu:
        torch.cuda.empty_cache()

使用 GPU.
系统中总共有 1 个 CUDA 设备可用。
使用单个 GPU.
Use GPU: cuda:0
>>>>>>>测试 : long_term_forecast_Stock_test_TimesNet_custom_ftMS_sl60_ll20_pl1_dm32_nh8_el2_dl1_df32_fc3_ebtimeF_dtTrue_stock_0<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


FileNotFoundError: [Errno 2] No such file or directory: './dataset/Stock/all_data_scaled.csv'

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./tensorboard_logs/